In [8]:
import torch
from PIL import Image
import torchvision
from torchvision import transforms
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
import torch.nn.functional as F
import random
from pathlib import Path
random.seed = 421

In [2]:
class Ageset(Dataset):

    def __init__(self, path, transforms = None, valid=False, split_pct = 0.3):
        self.image_paths = list(Path(path).rglob("*.png"))
        random.shuffle(self.image_paths)
        split_point = int(len(self)*0.3)
        if valid:
            self.image_paths = self.image_paths[:split_point]
        else:
            self.image_paths = self.image_paths[split_point:]
    def __len__(self):
        return len(self.image_paths)

    def imgpath_to_tensor(self, imgpath):
        return transforms.PILToTensor()(Image.open(imgpath)).float()

    def __getitem__(self,i):
        # return self.image_paths[i]
        return (self.imgpath_to_tensor(self.image_paths[i]),
                int(self.image_paths[i].parent.name))

In [3]:
class Model(nn.Module):
    def __init__(self):
        super(Model,self).__init__()
        self.conv1 = nn.Conv2d(3,20,3)
        self.conv2 = nn.Conv2d(20,20,3)
        self.fc = nn.Linear(768320,1)
    def forward(self, x):
        out = F.relu(self.conv1(x))
        out = F.relu(self.conv2(out))
        return self.fc(out.view(-1,768320))

In [24]:
class adaptedRes(nn.Module):
    def __init__(self):
        super(adaptedRes, self).__init__()
        resnet18 = torchvision.models.resnet18(pretrained=True)
        modules=list(resnet18.children())[:-1]
        self.resnet =nn.Sequential(*modules)
        self.fc = nn.Linear(in_features=512, out_features=1, bias=True)

        for m in self.modules():
            if isinstance(m, nn.Linear):
                nn.init.xavier_normal_(m.weight, gain = 1)

    def forward(self,x):
        out = self.resnet(x)
        x = torch.flatten(out, 1)
        return self.fc(x)

In [30]:
NUM_EPOCH = 5
LR = 0.01
BATCH_SIZE = 256
DEVICE = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

train_set = Ageset("data/face_age")
test_set = Ageset("data/face_age", valid=True)
print(len(train_set), len(test_set))
train_dl = DataLoader(train_set, batch_size=BATCH_SIZE, shuffle=True, num_workers=0)
test_dl = DataLoader(test_set, batch_size=BATCH_SIZE, shuffle=True, num_workers=0)
model = adaptedRes()
model.to(DEVICE)
opt = torch.optim.Adam(model.parameters(), LR)

def mae_loss(y, pred):
    return torch.mean(torch.abs(y-pred))

loss_fn = mae_loss


for epoch in range(NUM_EPOCH):
    print(epoch)
    for data in train_dl:
        x, y = data[0].to(DEVICE), data[1].to(DEVICE)
        opt.zero_grad()
        pred = model(x)
        loss = loss_fn(y, pred)
        loss.backward()
        opt.step()
    running_loss = 0
    with torch.no_grad():
        for data in test_dl:
            x, y = data[0].to(DEVICE), data[1].to(DEVICE)
            running_loss += loss_fn(y, model(x))
    print(running_loss/len(test_set)*BATCH_SIZE)
    


13690 5866
0
tensor(20.5781, device='cuda:0')
1
tensor(20.5341, device='cuda:0')
2
tensor(20.5826, device='cuda:0')
3


KeyboardInterrupt: 